In [1]:
import pandas as pd
import numpy as np
import requests
from PyPDF2 import PdfFileReader
import zipfile
from io import BytesIO
from pathlib import Path
from sklearn import preprocessing, model_selection, metrics
from tensorflow import keras

# Lectura y unificación de datos

En primer lugar indico la información relativa a cada tipo de datos para automatizar la extracción. Esto permite la adición de nuevos ficheros sin tener que cambiar el código.

In [2]:
data_csv = {
    'train1': {
        'path':'https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train1.csv',
        'sep': ','
    },
    'train2':{
        'path':'https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train2.csv',
        'sep': ';',
    }
}

data_json = {
    'train3': 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first',
    'train4': 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second',
    'train5': 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third',
}

url_pdf_zip = 'https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train6.zip'

Ahora defino una función para leer cada tipo de dato, estas funciones crean un dataframe para cada tipo de dato y se concatenarán en los próximos pasos.

In [3]:
def read_csv_data(data_csv):
    df = pd.DataFrame()
    for name, csv in data_csv.items():
        df_csv = pd.read_csv(csv['path'], sep=csv['sep'])
        df = pd.concat([df, df_csv], axis=0)
    return df

def read_json_data(data_json):
    df = pd.DataFrame()
    for name, endpoint in data_json.items():
        response = requests.get(endpoint)
        df_json = pd.json_normalize(response.json())
        df = pd.concat([df, df_json], axis=0)
    return df

def read_pdf_data(url):
    
    df = pd.DataFrame(columns = ['nº', 'facilityname', 'facilityinspireid', 'country', 'continent', 'city', 'eprtrsectorcode', 'eprtrsectorname', 'mainactivitycode', 'targetrealase', 'pollutant', 'emissions', 'day', 'month', 'year', 'max_wind_speed', 'min_wind_speed', 'avg_wind_speed', 'max_temp', 'min_temp', 'avg_temp', 'daysfog', 'reporter name', 'city_id'])
    
    # Descargo fichero enviando petición a la URL
    req = requests.get(url)
    # Extraigo el contenido del zip
    files= zipfile.ZipFile(BytesIO(req.content))
    files.extractall('../data/')
    # Selecciono todos los archivos de la carpeta train6
    files = Path('../data/train6').glob('*')
    # Leo cada fichero correspondiente a un PDF
    for file in files:
        reader = PdfFileReader(str(file))
        page = reader.pages[0]
        text = page.extractText()
        # El texto sale con espacios raros, separo por salto de línea, lo pongo todo el minúscula, elimino los espacios y elimino :
        text = text.lower().split('\n')
        text = [txt.replace(' ', '').replace(':','') for txt in text]
        # También elimino algunos campos que contiene el informe y que no me interesan
        pdf_data = [txt for txt in text if txt not in ['reportcontaminación', 'meteorologicalconditions', '']]
        
        if len(pdf_data)%2 != 0:
            pdf_data.pop(4)
        # Por cada dato par (nombre de columna) creo un diccionario con el nombre de la columna y el valor
        d = {}
        for i, txt in enumerate(pdf_data):
            if i%2 == 0:
                d[txt] = pdf_data[i+1]
        # Estos diccionarios los añado como filas a un dataframe   
        df = pd.concat([df, pd.DataFrame.from_records([d])])
        
    return df

Los datos extrídos de los pdfs tienen nombres de columnas distintos, todos los datos en minúsculas y sin espacios.

A continuación se mapean los nombres y valores de las columnas con los del resto de datos para unificar el formato.

In [4]:
column_map = {
    'facilityName': 'facilityname',
    'FacilityInspireID': 'facilityinspireid',
    'countryName': 'country',
    'CONTINENT': 'continent',
    'City': 'city',
    'EPRTRSectorCode': 'eprtrsectorcode',
    'eprtrSectorName': 'eprtrsectorname',
    'EPRTRAnnexIMainActivityCode': 'mainactivitycode',
    'targetRelease': 'targetrealase',
    'pollutant': 'pollutant',
    'DAY': 'day',
    'MONTH': 'month',
    'reportingYear': 'year',
    'max_wind_speed': 'max_wind_speed',
    'min_wind_speed': 'min_wind_speed',
    'avg_wind_speed': 'avg_wind_speed',
    'max_temp': 'max_temp',
    'min_temp': 'min_temp',
    'avg_temp': 'avg_temp',
    'DAY WITH FOGS': 'daysfog',
    'REPORTER NAME': 'reportername',
    'CITY ID': 'city_id',
}

In [5]:
def transform_str(x,d):
    try:
        return d[x]
    except:
        return x

def transform_num(x):
    x = x.replace(',','.')
    x = float(x.split('e+')[0]) * 10**int(x.split('e+')[1])
    return x

def clean_pdf_train(df_pdf, df_train):
    # Columnas textuales a las que hay que cambiar nombre y valor
    str_cols = ['countryName', 'eprtrSectorName', 'facilityName', 'City', 'targetRelease',
                'pollutant', 'CONTINENT', 'REPORTER NAME', 'CITY ID']
    # Columnas numéricas que transformamos de forma que no tengan 'e+n' si no el valor numérico
    num_cols = ['max_wind_speed', 'avg_wind_speed', 'min_wind_speed', 'max_temp', 'avg_temp', 'min_temp']
    # A estas columnas simplemente les cambio el nombre
    other_cols = ['FacilityInspireID', 'reportingYear', 'MONTH', 'DAY', 'DAY WITH FOGS', 'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode']
    
    df = pd.DataFrame()
    
    for col in str_cols:
        # Identifico el nombre de la columna en el dataframe de los pdfs
        pdf_col = column_map[col]
        # Guardo todos los valores únicos que toma la columna
        unique_values = list(df_train[col].unique())
        d = {}
        # Sustituyo valores de los pdfs (minúscula y sin espacios) con los que deberían tomar
        for val in unique_values:
            d[val.lower().replace(' ', '')] = val
        df[col] = df_pdf[pdf_col].apply(lambda x: transform_str(x,d))
        
    for col in num_cols:
        pdf_col = column_map[col]
        df[col]  = df_pdf[pdf_col].apply(lambda x: transform_num(x))
        
    for col in other_cols:
        df[col] = df_pdf[column_map[col]]
        
    return df

In [6]:
# Leo ficheros csv y json
df_csv_train = read_csv_data(data_csv)
df_json_train = read_json_data(data_json)
# Leo pdfs
df_pdf_train = read_pdf_data(url_pdf_zip)
# Uno los dataframes de los csv y json
df_train = pd.concat([df_csv_train, df_json_train.drop('', axis=1)], axis=0)
# Cambio el dataframe de los pdfs a partir de los valores de los csv y json
df_pdf = clean_pdf_train(df_pdf_train, df_train)
# Combino todos los datos para obtener el dataframe final
df_train = pd.concat([df_pdf, df_train], axis=0)

In [7]:
df_train

,countryName,eprtrSectorName,facilityName,City,targetRelease,pollutant,CONTINENT,REPORTER NAME,CITY ID,max_wind_speed,...,avg_temp,min_temp,FacilityInspireID,reportingYear,MONTH,DAY,DAY WITH FOGS,EPRTRAnnexIMainActivityCode,EPRTRSectorCode,EPRTRAnnexIMainActivityLabel
0,United Kingdom,Waste and wastewater management,Millerhill Recycling & Energy Recovery Centre,"Millerhill, Dalkeith",AIR,Nitrogen oxides (NOX),EUROPE,William Nelson,c662b4b4d859a9c224b5ac0acf221748,1790000000000000.0,...,17100000000000000.0,1820000000000000.0,uk.sepa/200002651.facility,2019,6,12,10,5(b),5,NaN
0,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,AIR,Carbon dioxide (CO2),EUROPE,shawngraham,3c563ab0d76fc84128574b5da82f769a,15200000000000000.0,...,8690000000000000.0,13300000000000000.0,uk.sepa/200000061.facility,2019,4,12,19,1(c),1,NaN
0,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,AIR,Carbon dioxide (CO2),EUROPE,shawngraham,3c563ab0d76fc84128574b5da82f769a,15200000000000000.0,...,8690000000000000.0,13300000000000000.0,uk.sepa/200000061.facility,2019,4,12,19,1(c),1,NaN
0,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,AIR,Nitrogen oxides (NOX),EUROPE,Aaron Johnson,3c563ab0d76fc84128574b5da82f769a,11600000000000000.0,...,8940000000000000.0,10400000000000000.0,uk.sepa/200000061.facility,2019,1,3,10,1(c),1,NaN
0,United Kingdom,Mineral industry,Alloa Glass Factory,Alloa,AIR,Carbon dioxide (CO2),EUROPE,vickibrown,2cc8f54182c37b8907f534011ea01e6f,11100000000000000.0,...,1330000000000000.0,40700000000000000.0,uk.sepa/200000073.facility,2019,11,9,4,3(e),3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9496,Cyprus,Energy sector,"Electricity Authority of Cyprus, Vassilikos Po...",LARNAKA,AIR,Carbon dioxide (CO2),EUROPE,Tammy Faulkner,2d4776365b33d5f1be53ea4606e2c79c,13.475987983553544,...,12.41078296998785,17.148326520153717,CY.CAED/0030030000.FACILITY,2008,1,1,0,1(c),1,Thermal power stations and other combustion in...
9497,Finland,Energy sector,"Turun Seudun Energiantuotanto Oy, Naantalin vo...",Naantali,AIR,Nitrogen oxides (NOX),EUROPE,Dr. Courtney Bryant,020b11bf06b96aae1dd910a56674a8aa,8.815938780886254,...,3.7638334614499307,5.657106835772357,http://paikkatiedot.fi/so/1002031/pf/Productio...,2008,12,19,0,1(c),1,Thermal power stations and other combustion in...
9498,Slovenia,Waste and wastewater management,"Javne službe Ptuj, Odlagališče nenevarnih odpa...",Ptuj,AIR,Methane (CH4),EUROPE,William Greer,84afdc8367dfd9124e8b8f994e986fe9,14.793298049775357,...,18.34979774493633,21.538440790883374,SI.ARSO/000000037.FACILITY,2010,8,10,2,5(d),5,Landfills (excluding landfills of inert waste ...
9499,Italy,Mineral industry,Centro Olio Val d'Agri,VIGGIANO,AIR,Nitrogen oxides (NOX),EUROPE,Leonard Roberts,09ad69bcf41256f40be3314a33e0438c,14.911317150406134,...,6.176237885779872,9.269075541047428,IT.CAED/850592002.FACILITY,2014,1,25,0,3(a),3,Underground mining and related operations


# Limpieza de datos

En este apartado voy a tratar de identificar las columnas innecesarias para eliminarlas.

In [8]:
df = df_train.reset_index(drop=True)

Veamos qué columnas contienen valores nulos y si a partir de los nombres podemos determinar cuáles aportan la misma información

In [9]:
df.isnull().sum()

countryName                         0
eprtrSectorName                     0
facilityName                        0
City                                0
targetRelease                       0
pollutant                           0
CONTINENT                           0
REPORTER NAME                       0
CITY ID                             0
max_wind_speed                      0
avg_wind_speed                      0
min_wind_speed                      0
max_temp                            0
avg_temp                            0
min_temp                            0
FacilityInspireID                   0
reportingYear                       0
MONTH                               0
DAY                                 0
DAY WITH FOGS                       0
EPRTRAnnexIMainActivityCode     37127
EPRTRSectorCode                 37127
EPRTRAnnexIMainActivityLabel       82
dtype: int64

A continuación se comprobará si:
```
facilityName == FacilityInspireID
EPRTRAnnexIMainActivityCode == EPRTRAnnexIMainActivityLabel
EPRTRSectorCode == eprtrSectorName
City == CITY ID
```
para ver si se pueden eliminar columnas innecesarias

In [10]:
len(df['facilityName'].unique())

7934

In [11]:
len(df['FacilityInspireID'].unique())

7246

Ahora se creará un diccionario de forma que cada nombre tenga una lista de ID asociados.

In [12]:
d = {}

for idx, row in df.iterrows():
    if row['facilityName'] not in d:
        d[row['facilityName']] = [row['FacilityInspireID']]
    elif row['FacilityInspireID'] in d[row['facilityName']]:
        pass
    else:
        d[row['facilityName']].append(row['FacilityInspireID'])

Veamos los 10 primeros para comprobar si cada nombre está relacionado con un único ID

In [13]:
list(d.items())[:10]

[('Millerhill Recycling & Energy Recovery Centre',
  ['uk.sepa/200002651.facility', 'UK.SEPA/200002651.Facility']),
 ('Fife Ethylene Plant',
  ['uk.sepa/200000061.facility', 'UK.SEPA/200000061.Facility']),
 ('Alloa Glass Factory',
  ['uk.sepa/200000073.facility', 'UK.SEPA/200000073.Facility']),
 ('Restenneth Landfill Site, Forfar',
  ['uk.sepa/200000081.facility', 'UK.SEPA/200000081.Facility']),
 ('Auchenlosh Landfill Site',
  ['uk.sepa/200000082.facility', 'UK.SEPA/200000082.Facility']),
 ('Auchencarroch Landfill, Jamestown',
  ['uk.sepa/200000083.facility', 'UK.SEPA/200000083.Facility']),
 ('Garlaff Landfill Site, Cumnock',
  ['uk.sepa/200000084.facility', 'UK.SEPA/200000084.Facility']),
 ('Binn Farm Landfill Site',
  ['uk.sepa/200000085.facility', 'UK.SEPA/200000085.Facility']),
 ('Kilgarth Landfill, Coatbridge',
  ['uk.sepa/200000086.facility', 'UK.SEPA/200000086.Facility']),
 ('blackdevonlandfillsite,alloa', ['uk.sepa/200000087.facility'])]

Comprobamos que coinciden, en este caso cada nombre tiene dos ID asociados pero son equivalentes ya que uno está en minúscula y el otro en mayúscula. Esto se debe seguramente a la transformación que se ha realizado para etxraer los datos de los pdfs.

Realizamos la misma comprobación para el resto de variables.

In [14]:
d = {}

for idx, row in df.iterrows():
    if row['eprtrSectorName'] not in d:
        d[row['eprtrSectorName']] = [row['EPRTRSectorCode']]
    elif row['EPRTRSectorCode'] in d[row['eprtrSectorName']]:
        pass
    else:
        d[row['eprtrSectorName']].append(row['EPRTRSectorCode'])
        
list(d.items())[:10]

[('Waste and wastewater management', ['5', nan]),
 ('Energy sector', ['1', nan]),
 ('Mineral industry', ['3', nan]),
 ('Paper and wood production and processing', ['6', nan]),
 ('Chemical industry', ['4', nan]),
 ('Production and processing of metals', [nan, '2']),
 ('Intensive livestock production and aquaculture', [nan, '7']),
 ('Animal and vegetable products from the food and beverage sector',
  [nan, '8']),
 ('Other activities', [nan, '9'])]

In [15]:
d = {}

for idx, row in df.iterrows():
    if row['City'] not in d:
        d[row['City']] = [row['CITY ID']]
    elif row['CITY ID'] in d[row['City']]:
        pass
    else:
        d[row['City']].append(row['CITY ID'])
        
list(d.items())[:10]

[('Millerhill, Dalkeith', ['c662b4b4d859a9c224b5ac0acf221748']),
 ('Cowdenbeath', ['3c563ab0d76fc84128574b5da82f769a']),
 ('Alloa', ['2cc8f54182c37b8907f534011ea01e6f']),
 ('Forfar', ['f66f865a6e8c90a9bcafc9def2e701f5']),
 ('Dalbeattie', ['d52bec466f0edb76a6fc14c109598ba2']),
 ('Alexandria', ['a6bc8f85a33112c5beea4d357caad4cd']),
 ('Cumnock', ['b6ca640b7121ae2bf206ea088f6a4618']),
 ('Glenfarg', ['930b3f6a2b01ab9967c61d939efce973']),
 ('Coatbridge', ['2b83cfb769c8c15a3a0329410129a67f']),
 ('alloa,clackmannanshire', ['10fac069cd96c396ab2b6998ae548389'])]

In [16]:
d = {}

for idx, row in df.iterrows():
    if row['EPRTRAnnexIMainActivityCode'] not in d:
        d[row['EPRTRAnnexIMainActivityCode']] = [row['EPRTRAnnexIMainActivityLabel']]
    elif row['EPRTRAnnexIMainActivityLabel'] in d[row['EPRTRAnnexIMainActivityCode']]:
        pass
    else:
        d[row['EPRTRAnnexIMainActivityCode']].append(row['EPRTRAnnexIMainActivityLabel'])
        
list(d.items())[:10]

[('5(b)',
  [nan,
   'Installations for the incineration of non-hazardous waste in the scope of Directive 2000/76/EC of the European Parliament and of the Council of 4 December 2000 on the incineration of waste']),
 ('1(c)', [nan, 'Thermal power stations and other combustion installations']),
 ('3(e)',
  [nan, 'Installations for the manufacture of glass, including glass fibre']),
 ('5(d)',
  [nan,
   'Landfills (excluding landfills of inert waste and landfills, which were definitely closed before 16.7.2001 or for which the after-care phase required by the competent authorities according to Article 13 of Council Directive 1999/31/EC of 26 April 1999 on the landfill of waste has expired)']),
 ('5(c)', [nan, 'Installations for the disposal of non-hazardous waste']),
 ('6(b)',
  [nan,
   'Industrial plants for the production of paper and board and other primary wood products (such as chipboard, fibreboard and plywood)']),
 ('1(a)', [nan, 'Mineral oil and gas refineries']),
 ('4(a)(ii)',
  

En el caso de estas variables observamos que también coinciden, por lo que una de cada pareja se podría eliminar.

En el caso de la última pareja (`EPRTRAnnexIMainActivityCode`, `EPRTRAnnexIMainActivityLabel`) existen valores nulos en ambas columnas, por lo tanto debemos asegurarnos de rellenar los valores nulos de `EPRTRAnnexIMainActivityLabel` antes de eliminar la columna `EPRTRAnnexIMainActivityCode`.

In [17]:
def label_per_code(x):
    label = d[x]
    if len(label) == 1:
        return label[0]
    elif len(label) == 2:
        label = [x for x in label if pd.isnull(x) == False]
        return label[0]
    else:
        return pd.NA   

In [18]:
df.loc[df['EPRTRAnnexIMainActivityLabel'].isnull(),'EPRTRAnnexIMainActivityLabel'] = df[df['EPRTRAnnexIMainActivityLabel'].isnull()]['EPRTRAnnexIMainActivityCode'].apply(lambda x: label_per_code(x))

In [19]:
df.isnull().sum()

countryName                         0
eprtrSectorName                     0
facilityName                        0
City                                0
targetRelease                       0
pollutant                           0
CONTINENT                           0
REPORTER NAME                       0
CITY ID                             0
max_wind_speed                      0
avg_wind_speed                      0
min_wind_speed                      0
max_temp                            0
avg_temp                            0
min_temp                            0
FacilityInspireID                   0
reportingYear                       0
MONTH                               0
DAY                                 0
DAY WITH FOGS                       0
EPRTRAnnexIMainActivityCode     37127
EPRTRSectorCode                 37127
EPRTRAnnexIMainActivityLabel        0
dtype: int64

In [20]:
df = df.drop(['FacilityInspireID', 'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode', 'CITY ID'], axis=1)

Veamos ahora cuántos valores únicos hay en cada columna, ya que si tan solo hay uno no aportará información relevante para el modelo.

In [21]:
for col in df.columns:
    print(col, df[col].nunique())

countryName 32
eprtrSectorName 9
facilityName 7934
City 5138
targetRelease 1
pollutant 3
CONTINENT 1
REPORTER NAME 45065
max_wind_speed 57130
avg_wind_speed 57122
min_wind_speed 57134
max_temp 57138
avg_temp 57133
min_temp 57131
reportingYear 28
MONTH 24
DAY 56
DAY WITH FOGS 40
EPRTRAnnexIMainActivityLabel 71


Se observa que las variables `targetRelease` y `CONTINENT` toman un único valor, por lo que a continuación se elimarán.

In [22]:
df = df.drop(['targetRelease', 'CONTINENT'], axis=1)

In [23]:
df

,countryName,eprtrSectorName,facilityName,City,pollutant,REPORTER NAME,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,reportingYear,MONTH,DAY,DAY WITH FOGS,EPRTRAnnexIMainActivityLabel
0,United Kingdom,Waste and wastewater management,Millerhill Recycling & Energy Recovery Centre,"Millerhill, Dalkeith",Nitrogen oxides (NOX),William Nelson,1790000000000000.0,2040000000000000.0,22000000000000000.0,15100000000000000.0,17100000000000000.0,1820000000000000.0,2019,6,12,10,Installations for the incineration of non-haza...
1,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,Carbon dioxide (CO2),shawngraham,15200000000000000.0,14600000000000000.0,2060000000000000.0,9610000000000000.0,8690000000000000.0,13300000000000000.0,2019,4,12,19,Thermal power stations and other combustion in...
2,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,Carbon dioxide (CO2),shawngraham,15200000000000000.0,14600000000000000.0,2060000000000000.0,9610000000000000.0,8690000000000000.0,13300000000000000.0,2019,4,12,19,Thermal power stations and other combustion in...
3,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,Nitrogen oxides (NOX),Aaron Johnson,11600000000000000.0,16500000000000000.0,21800000000000000.0,8029999999999999.0,8940000000000000.0,10400000000000000.0,2019,1,3,10,Thermal power stations and other combustion in...
4,United Kingdom,Mineral industry,Alloa Glass Factory,Alloa,Carbon dioxide (CO2),vickibrown,11100000000000000.0,16000000000000000.0,20300000000000000.0,-19000000000000000.0,1330000000000000.0,40700000000000000.0,2019,11,9,4,"Installations for the manufacture of glass, in..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65705,Cyprus,Energy sector,"Electricity Authority of Cyprus, Vassilikos Po...",LARNAKA,Carbon dioxide (CO2),Tammy Faulkner,13.475987983553544,18.55647637777159,22.85253008379323,13.34580070979159,12.41078296998785,17.148326520153717,2008,1,1,0,Thermal power stations and other combustion in...
65706,Finland,Energy sector,"Turun Seudun Energiantuotanto Oy, Naantalin vo...",Naantali,Nitrogen oxides (NOX),Dr. Courtney Bryant,8.815938780886254,14.461702597343331,20.55378092877372,3.820281499959616,3.7638334614499307,5.657106835772357,2008,12,19,0,Thermal power stations and other combustion in...
65707,Slovenia,Waste and wastewater management,"Javne službe Ptuj, Odlagališče nenevarnih odpa...",Ptuj,Methane (CH4),William Greer,14.793298049775357,16.688049243796055,20.411497765443265,17.28536462543226,18.34979774493633,21.538440790883374,2010,8,10,2,Landfills (excluding landfills of inert waste ...
65708,Italy,Mineral industry,Centro Olio Val d'Agri,VIGGIANO,Nitrogen oxides (NOX),Leonard Roberts,14.911317150406134,16.1440911999443,22.64719217689789,6.387198549855581,6.176237885779872,9.269075541047428,2014,1,25,0,Underground mining and related operations


Observamos también que existen las columnas de día, mes y año que se podrían unir para crear una única columna que incluyera la fecha.

In [24]:
df = df.rename(columns={"reportingYear":"YEAR"})
df['Datetime'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

In [25]:
df = df.drop(['YEAR', 'MONTH', 'DAY'], axis=1)

In [26]:
df

,countryName,eprtrSectorName,facilityName,City,pollutant,REPORTER NAME,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,EPRTRAnnexIMainActivityLabel,Datetime
0,United Kingdom,Waste and wastewater management,Millerhill Recycling & Energy Recovery Centre,"Millerhill, Dalkeith",Nitrogen oxides (NOX),William Nelson,1790000000000000.0,2040000000000000.0,22000000000000000.0,15100000000000000.0,17100000000000000.0,1820000000000000.0,10,Installations for the incineration of non-haza...,2019-06-12
1,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,Carbon dioxide (CO2),shawngraham,15200000000000000.0,14600000000000000.0,2060000000000000.0,9610000000000000.0,8690000000000000.0,13300000000000000.0,19,Thermal power stations and other combustion in...,2019-04-12
2,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,Carbon dioxide (CO2),shawngraham,15200000000000000.0,14600000000000000.0,2060000000000000.0,9610000000000000.0,8690000000000000.0,13300000000000000.0,19,Thermal power stations and other combustion in...,2019-04-12
3,United Kingdom,Energy sector,Fife Ethylene Plant,Cowdenbeath,Nitrogen oxides (NOX),Aaron Johnson,11600000000000000.0,16500000000000000.0,21800000000000000.0,8029999999999999.0,8940000000000000.0,10400000000000000.0,10,Thermal power stations and other combustion in...,2019-01-03
4,United Kingdom,Mineral industry,Alloa Glass Factory,Alloa,Carbon dioxide (CO2),vickibrown,11100000000000000.0,16000000000000000.0,20300000000000000.0,-19000000000000000.0,1330000000000000.0,40700000000000000.0,4,"Installations for the manufacture of glass, in...",2019-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65705,Cyprus,Energy sector,"Electricity Authority of Cyprus, Vassilikos Po...",LARNAKA,Carbon dioxide (CO2),Tammy Faulkner,13.475987983553544,18.55647637777159,22.85253008379323,13.34580070979159,12.41078296998785,17.148326520153717,0,Thermal power stations and other combustion in...,2008-01-01
65706,Finland,Energy sector,"Turun Seudun Energiantuotanto Oy, Naantalin vo...",Naantali,Nitrogen oxides (NOX),Dr. Courtney Bryant,8.815938780886254,14.461702597343331,20.55378092877372,3.820281499959616,3.7638334614499307,5.657106835772357,0,Thermal power stations and other combustion in...,2008-12-19
65707,Slovenia,Waste and wastewater management,"Javne službe Ptuj, Odlagališče nenevarnih odpa...",Ptuj,Methane (CH4),William Greer,14.793298049775357,16.688049243796055,20.411497765443265,17.28536462543226,18.34979774493633,21.538440790883374,2,Landfills (excluding landfills of inert waste ...,2010-08-10
65708,Italy,Mineral industry,Centro Olio Val d'Agri,VIGGIANO,Nitrogen oxides (NOX),Leonard Roberts,14.911317150406134,16.1440911999443,22.64719217689789,6.387198549855581,6.176237885779872,9.269075541047428,0,Underground mining and related operations,2014-01-25


# Preprocesamiento de los datos

In [27]:
df_process = df

In [28]:
df_process['pollutant'] = df['pollutant'].map({
    'Nitrogen oxides (NOX)': 0,
    'Carbon dioxide (CO2)': 1,
    'Methane (CH4)': 2,
})

In [29]:
y = df_process['pollutant'].values
df_process = df_process.drop('pollutant', axis=1)

In [30]:
df_process.columns

Index(['countryName', 'eprtrSectorName', 'facilityName', 'City',
       'REPORTER NAME', 'max_wind_speed', 'avg_wind_speed', 'min_wind_speed',
       'max_temp', 'avg_temp', 'min_temp', 'DAY WITH FOGS',
       'EPRTRAnnexIMainActivityLabel', 'Datetime'],
      dtype='object')

In [31]:
categorical_columns = ['countryName', 'eprtrSectorName', 'facilityName', 'City', 'REPORTER NAME', 'EPRTRAnnexIMainActivityLabel']

for cat in categorical_columns:

    label_encoder = preprocessing.LabelEncoder()
    df_process[cat] = label_encoder.fit_transform(df_process[cat])
    
df_process['Datetime'] = df_process['Datetime'].values.astype(np.int64) // 10 ** 9

In [32]:
df_process.head()

,countryName,eprtrSectorName,facilityName,City,REPORTER NAME,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,EPRTRAnnexIMainActivityLabel,Datetime
0,31,8,4564,2865,44593,1790000000000000.0,2040000000000000.0,22000000000000000.0,15100000000000000.0,17100000000000000.0,1820000000000000.0,10,35,1560297600
1,31,2,2904,1105,45058,15200000000000000.0,14600000000000000.0,2060000000000000.0,9610000000000000.0,8690000000000000.0,13300000000000000.0,19,64,1555027200
2,31,2,2904,1105,45058,15200000000000000.0,14600000000000000.0,2060000000000000.0,9610000000000000.0,8690000000000000.0,13300000000000000.0,19,64,1555027200
3,31,2,2904,1105,70,11600000000000000.0,16500000000000000.0,21800000000000000.0,8029999999999999.0,8940000000000000.0,10400000000000000.0,10,64,1546473600
4,31,4,433,216,45063,11100000000000000.0,16000000000000000.0,20300000000000000.0,-19000000000000000.0,1330000000000000.0,40700000000000000.0,4,42,1573257600


In [33]:
scaler = preprocessing.MinMaxScaler()

x = scaler.fit_transform(df_process)

In [34]:
x

array([[1.        , 1.        , 0.57531829, ..., 0.52631579, 0.5       ,
        0.88943249],
       [1.        , 0.25      , 0.3660658 , ..., 1.        , 0.91428571,
        0.87749511],
       [1.        , 0.25      , 0.3660658 , ..., 1.        , 0.91428571,
        0.87749511],
       ...,
       [0.87096774, 1.        , 0.4807765 , ..., 0.10526316, 0.82857143,
        0.25772994],
       [0.48387097, 0.5       , 0.20988277, ..., 0.        , 0.98571429,
        0.50508806],
       [1.        , 0.25      , 0.7862095 , ..., 0.68421053, 0.91428571,
        0.11135029]])

# Entrenamiento del modelo

In [35]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, random_state=42)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
num_classes = 3

In [36]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 14, 64)            256       
                                                                 
 batch_normalization (BatchN  (None, 14, 64)           256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 14, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 64)            12352     
                                                                 
 batch_normalization_1 (Batc  (None, 14, 64)           256       
 hNormalization)                                             

In [38]:
epochs = 500
batch_size = 5

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/500
9857/9857 [==============================] - 35s 3ms/step - loss: 0.9324 - sparse_categorical_accuracy: 0.5169 - lr: 0.0010
Epoch 2/500
9857/9857 [==============================] - 35s 4ms/step - loss: 0.8467 - sparse_categorical_accuracy: 0.5454 - lr: 0.0010
Epoch 3/500
9857/9857 [==============================] - 34s 3ms/step - loss: 0.8120 - sparse_categorical_accuracy: 0.5634 - lr: 0.0010
Epoch 4/500
9857/9857 [==============================] - 36s 4ms/step - loss: 0.7940 - sparse_categorical_accuracy: 0.5693 - lr: 0.0010
Epoch 5/500
9857/9857 [==============================] - 37s 4ms/step - loss: 0.7811 - sparse_categorical_accuracy: 0.5732 - lr: 0.0010
Epoch 6/500
9857/9857 [==============================] - 38s 4ms/step - loss: 0.7714 - sparse_categorical_accuracy: 0.5778 - lr: 0.0010
Epoch 7/500
9857/9857 [==============================] - 34s 3ms/step - loss: 0.7664 - sparse_categorical_accuracy: 0.5800 - lr: 0.0010
Epoch 8/500
9857/9857 [=========================

KeyboardInterrupt: 

He interrumpido manualmente el entrenamiento porque no me daba tiempo a finalizarlo

In [39]:
pred = model.predict(x_test)
res = []
for p in pred:
    res.append(p.argmax())
print('The accuracy score on the test set is %.2f'%metrics.accuracy_score(res,y_test))

514/514 [==============================] - 1s 2ms/step
The accuracy score on the test set is 0.63


In [40]:
print(metrics.classification_report(res,y_test))

              precision    recall  f1-score   support

           0       0.54      0.56      0.55      6304
           1       0.53      0.52      0.52      5918
           2       0.88      0.88      0.88      4206

    accuracy                           0.63     16428
   macro avg       0.65      0.65      0.65     16428
weighted avg       0.63      0.63      0.63     16428



# Predictions

In [41]:
df_test = pd.read_csv('https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/test_x.csv')

In [42]:
df_test.head()

,test_index,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,...,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,0,Poland,3,Mineral industry,3(a),Underground mining and related operations,PL.MŚ/000002357.FACILITY,Polska Grupa Górnicza sp. z o.o. Oddział KWK R...,Rydułtowy,AIR,...,EUROPE,14.080054,14.855940,18.474836,10.278561,11.381181,13.480752,1,Brittany Buck,826b1de9dad293ae3e4f9cbaf6cf3420
1,1,Luxembourg,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,LU.CAED/000012000.FACILITY,Sidec,Diekirch,AIR,...,EUROPE,16.052425,17.623877,22.622900,6.625910,8.840137,13.422924,0,Lauren Fisher,ed30a6667b40ba0a66198b3173e7353f
2,2,Netherlands,1,Energy sector,1(c),Thermal power stations and other combustion in...,NL.EEA/212857.FACILITY,Nuon Power Generation BV (Eemshaven),Eemshaven,AIR,...,EUROPE,13.647318,15.541979,17.818521,5.669436,8.403322,11.276388,2,Linda Thompson,78e1082c3cfef3bdf3554da8d6afcc34
3,3,Sweden,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,SE.CAED/10013901.Facility,HÖGBYTORPS AVFALLSANLÄGGNING,BRO,AIR,...,EUROPE,16.336762,17.458113,19.961777,6.160560,7.571596,9.443572,2,Bethany Mcmillan,27f959641950d381869d746d7d0e7d4e
4,4,Portugal,1,Energy sector,1(c),Thermal power stations and other combustion in...,PT.EEA/133926.FACILITY,SPCG - Sociedade Portuguesa de Co-Geração Eléc...,SETÚBAL,AIR,...,EUROPE,21.516602,20.532473,21.617137,10.964012,11.548033,12.624114,2,Sarah Hoffman,1cb71655d9e0bd5cedb2320bf5fdd8f7


Aplicamos el mismo preprocesamiento a los datos de test que a los de entrenamiento

In [43]:
predictions = pd.DataFrame()
predictions['test_index'] = df_test['test_index']

In [44]:
df_test = df_test.rename(columns={"reportingYear":"YEAR"})
df_test['Datetime'] = pd.to_datetime(df_test[['YEAR', 'MONTH', 'DAY']])

df_test = df_test.drop(['test_index', 'FacilityInspireID', 'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode', 'CITY ID', 'targetRelease', 'CONTINENT', 'YEAR', 'MONTH', 'DAY'], axis=1)

In [45]:
df_all = pd.concat([df.drop('pollutant', axis=1), df_test], axis=0)

In [46]:
categorical_columns = ['countryName', 'eprtrSectorName', 'facilityName', 'City', 'REPORTER NAME', 'EPRTRAnnexIMainActivityLabel']

for cat in categorical_columns:

    label_encoder = preprocessing.LabelEncoder()
    df_all[cat] = label_encoder.fit_transform(df_all[cat])
    
df_all['Datetime'] = df_all['Datetime'].values.astype(np.int64) // 10 ** 9

In [47]:
scaler = preprocessing.MinMaxScaler()

x = scaler.fit_transform(df_all)

In [48]:
x_test = x[-len(df_test):]
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [49]:
pred = model.predict(x_test)

765/765 [==============================] - 2s 3ms/step


In [50]:
predictions['pollutant'] = pred.argmax(axis=1)

In [51]:
predictions

,test_index,pollutant
0,0,2
1,1,2
2,2,1
3,3,2
4,4,0
...,...,...
24475,24475,2
24476,24476,1
24477,24477,2
24478,24478,1


In [54]:
predictions.to_csv('predictions.csv', index=False)
predictions.to_json('predictions.json')